In [1]:
import os
import subprocess
import gzip
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import random

from datasets.protein import PfamDataset
from encoder.protein_encoders import ProteinSetEncoder
from generator.protein_generator import Progen2Generator

from torch.utils.data import DataLoader
import torch

In [2]:
# data_dir = "data/pfam"
# os.makedirs(data_dir, exist_ok=True)

# # download regions.tsv.gz files
# url = "ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/"
# wget_cmd = f"wget -nd -r -A '*.fasta.gz' -P {data_dir} {url}"
# subprocess.run(wget_cmd, shell=True)

In [3]:
pf = PfamDataset(tokenize=False)

In [4]:
dataloader = DataLoader(pf, batch_size=1)
encoder = ProteinSetEncoder(latent_dim=16).to(device='cuda')
generator = Progen2Generator(latent_dim=16, max_length=512).to(device='cuda')

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
for batch in dataloader:
    samples = {}
    for key, value in batch['samples'].items():
        if isinstance(value, torch.Tensor):
            samples[key] = value.to('cuda')
        else:
            samples[key] = value

    print(samples['esm_input_ids'].shape)
    print(samples['progen_input_ids'].shape)

    latent = encoder(samples)
    print(latent.shape)

    input_ids = samples['progen_input_ids']

    bs, set_size, seq_len = input_ids.shape
    print("input ids shape", input_ids.shape)
    
    input_ids = input_ids.view(bs * set_size, seq_len)
    shift_logits = generator.model(input_ids, latent)[:, :-1, :]
    loss = generator.loss(samples, latent)
    print(loss)
    break

torch.Size([1, 16, 512])
torch.Size([1, 16, 512])
torch.Size([1, 16])
input ids shape torch.Size([1, 16, 512])
prefix emb shape torch.Size([16, 10, 1536])
input emb shape torch.Size([16, 512, 1536])
input ids shape torch.Size([1, 16, 512])
prefix emb shape torch.Size([16, 10, 1536])
input emb shape torch.Size([16, 512, 1536])


OutOfMemoryError: CUDA out of memory. Tried to allocate 196.00 MiB. GPU 0 has a total capacity of 79.10 GiB of which 68.75 MiB is free. Including non-PyTorch memory, this process has 79.02 GiB memory in use. Of the allocated memory 76.88 GiB is allocated by PyTorch, and 1.49 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
shift_logits.shape

torch.Size([16, 510, 32])

In [ ]:
input_ids.view(1*16, 512)[:, 1:].shape

torch.Size([16, 511])

In [ ]:
shift_logits.shape

torch.Size([16, 510, 32])